# Simulation of the Process with Process Variations

Import PyRolL modules.

In [2]:
import pyroll.basic as pr

Import the process data.

In [17]:
from weiner_variation.sim.process import PASS_SEQUENCE, IN_PROFILE, TEMPERATURE, DIAMETER, TEMPERATURE_STD, DIAMETER_STD, create_in_profile
from weiner_variation.sim.config import FIELDS, SAMPLE_COUNT, SEED
from weiner_variation.sim.data_structures import DrawDurations
from weiner_variation.config import DATA_DIR

Import further libraries.

In [14]:
import pandas as pd
from copy import deepcopy
from scipy.stats import norm, weibull_min
import numpy as np
import tqdm
from multiprocessing import Pool

Parameters for notebook using papermill.

In [11]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = "sim_durations_results.csv"

Load pause duration data in a dataframe.

In [12]:
df_durations = pd.read_csv(DATA_DIR / "duo_pauses_dist.csv", header=0, index_col=0)
df_durations

,mean,median,std,min,max,shape,scale,fit_mean,fit_std
R1-R2,6.633088,6.4000,1.471388,5.115,15.955,8.212077,6.543489,6.170061,0.893318
R2-R3,4.435441,4.2500,0.995052,3.040,11.100,7.705520,4.407880,4.143458,0.636670
R3-R4,5.988015,5.6100,2.192780,4.015,21.000,7.029720,5.623389,5.261487,0.880575
R4-R5,5.654853,5.6625,0.470526,4.745,7.115,13.799551,5.710760,5.499720,0.487191
R5-R6,5.044559,5.0200,0.566902,3.635,6.030,8.707140,5.276168,4.988786,0.683739
R6-R7,5.195588,5.0850,0.588969,4.235,8.095,16.114621,5.085251,4.921435,0.375704
R7-R8,6.435882,6.1100,1.557230,4.705,13.170,8.146553,6.161134,5.807295,0.847121
R8-R9,6.080735,5.8800,0.723092,4.745,8.260,14.631003,5.900644,5.693526,0.476881
R9-R10,7.008235,7.0050,0.775194,5.260,9.215,11.127373,7.139707,6.822094,0.741851
R10-F1,9.233309,9.1775,0.399014,8.290,10.385,37.735730,9.203396,9.068859,0.302529


Create distribution functions for input values.

In [15]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)
durations_dists = [weibull_min(c=r["shape"], scale=r["scale"]) for i, r in df_durations.iterrows()]

Draw random inputs from distributions.

In [18]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
durations = np.concatenate([
    d.rvs(random_state=RNG, size=(SAMPLE_COUNT, 1))
    for d in durations_dists[:-1]
], axis=1)

draws = [
    DrawDurations(d, t, dur)
    for d, t, dur in zip(diameters, temperatures, durations)
]

Define a worker function creating the in profile, running the solution procedure and extracting results.

In [22]:
def worker(draw: DrawDurations):    
    ip = create_in_profile(draw.diameter)
    ip.temperature = draw.temperature

    sequence = deepcopy(PASS_SEQUENCE)  
    
    transports = [u for u in sequence if isinstance(u, pr.Transport)]
    for t, d in zip(transports, draw.durations):
        t.duration = d
    
    sequence.solve(ip)       

    return (
        draw.__dict__.items() |> map$(t -> (("draw", t[0]), t[1])) |> dict
    ) | (
        FIELDS.items()
        |> starmap$((key, extractor) -> (
                sequence.units 
                |> filter$(u -> isinstance(u, pr.RollPass)) 
                |> map$(u -> ((key, u.label), extractor(u))) 
                |> filter$(t -> t[1] is not None)
            )
        )
        |> flatten
        |> dict            
    )

Run the simulations using a process pool.

In [23]:
results = tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT) |> list

100%|██████████| 10/10 [00:01<00:00,  5.31it/s]


Create a dataframe from results.

In [25]:
df = results |> filter$(e -> "error" not in e) |> enumerate |> dict |> pd.DataFrame |> .T |> .infer_objects()
df

draw                                                                  \
   diameter  temperature                                          durations   
0  0.050390  1433.393000  [6.723683817795442, 4.437947238026818, 5.92022...   
1  0.050772  1413.231456  [6.48238081333903, 3.708503178965375, 6.549300...   
2  0.051325  1438.005183  [6.90907440482713, 3.4469969751061704, 3.74331...   
3  0.049996  1417.283792  [6.328591499545676, 3.4547092498593552, 4.2507...   
4  0.050787  1412.266876  [6.7279486648728195, 3.7331187833247705, 6.635...   
5  0.051227  1425.030163  [6.703287729591661, 4.567633940131921, 5.11723...   
6  0.050200  1433.518216  [5.58504731423822, 4.803190362622017, 5.793046...   
7  0.050152  1422.000784  [7.173666195352142, 4.831937545615116, 5.17875...   
8  0.048080  1422.537161  [5.895563120655166, 4.365454705504861, 4.21415...   
9  0.049793  1416.863055  [6.826654169927619, 3.8894617697878293, 5.5009...   

      roll_force                                                              \
              R1             R2             R3             R4             R5   
0  300544.603264  174872.003267  311897.355526  144842.003992  193826.440815   
1  320084.580270  183035.326853  321030.727246  149195.617698  200797.884633   
2  311456.759557  177254.533515  309248.524702  139313.172823  187285.631172   
3  305225.921663  178390.789882  316317.021365  144296.075066  196048.791122   
4  320994.360937  183790.158258  322176.509387  149781.683233  200877.719210   
5  318925.026530  181046.061812  318211.920176  145305.864293  196307.055768   
6  297602.091391  172490.320335  309850.026726  143983.544515  195505.746087   
7  304471.413880  178523.457792  319336.872861  146890.018856  199311.195390   
8  272668.808873  169041.093548  310603.843114  143920.760187  193484.969049   
9  302355.265379  178501.956368  318263.661341  147337.602892  199174.848614   

                                 ... out_recrystallized_fraction              \
              R6             R7  ...                          R5 R6 R7 R8 R9   
0   97967.093440  182773.642559  ...                           0  0  0  0  0   
1  100049.855498  187068.759674  ...                           0  0  0  0  0   
2   94743.686444  178510.782896  ...                           0  0  0  0  0   
3   98410.599346  184274.139422  ...                           0  0  0  0  0   
4  100572.360142  187238.170288  ...                           0  0  0  0  0   
5   97269.559184  181519.608273  ...                           0  0  0  0  0   
6   97507.695748  183583.435906  ...                           0  0  0  0  0   
7   99963.264584  186575.450024  ...                           0  0  0  0  0   
8   95799.039924  180957.472742  ...                           0  0  0  0  0   
9   99233.744348  185833.893327  ...                           0  0  0  0  0   

                   
  R10 F1 F2 F3 F4  
0   0  0  0  0  0  
1   0  0  0  0  0  
2   0  0  0  0  0  
3   0  0  0  0  0  
4   0  0  0  0  0  
5   0  0  0  0  0  
6   0  0  0  0  0  
7   0  0  0  0  0  
8   0  0  0  0  0  
9   0  0  0  0  0  

[10 rows x 185 columns]

Save dataframe to CSV file.

In [26]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)